## RAG의 문서 검색기, Retriever    
RAG에 활용할 문서를 저장하고 시스템에 결합 가능한 형태로 가공하는 단계를 거친 후 사용자의 질문과 근거 문서를 잘 연결할 지의 단계   
RAG기반에서 아주 중요한 단계      
    - 사용자의 질문을 어떻게 해석할것인가 / 사용자 행동 예측         
      - 사용자가 저장된 문서의 문장과 유사한 질문을 할 때는 문제없으나 문서에 없는 문장, 즉 문장 유사도로 근거를 찾을 수 없는 표현으로 질문을 할 때는?     
    - 답변 근거가 될 문서를 어떻게 얼마나 가져올 것인가          
      - RAG은 사용자의 질문과 유사한 청크를 벡터 DB 검색으로 찾는데 유사 청크를 가져올 때 몇 개를 가져올지 정해야 한다.           
        - 모든 청크를 가져오면 컨텍스트 윈도우 초과, 어떤 부분을 집중적으로 참고할 지 몰라서 답변 품질 저하 / 유사도 순위 별로 하는게 답변의 품질을 보장하지 못한다. 근거를 포함하도록 하는 것이 좋다.   
이런 고려 사항들을 감안한 최적 파라미터 검색 단계 -> Retriever(검색기) 

In [ ]:
# 벡터 DB 기반의 Retriever 
# 가장 기초 형태
# 벡터 DB는 문장 간 임베딩 유사도 계산 기능이 있어 랭체인 결합 없이도 검색 기능을 구축할 수 있으나 랭체인의 Retriever 모듈로 더 쉽고 세밀한 검색 기능을 만들 수 있다. 

from langchain.document_loaders import PyPDFLoader 
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma 
import os 

api_key = os.environ["GEMINI_API_KEY"]

Chroma().delete_collection()

# 헌법 PDF 파일 로드
loader = PyPDFLoader("./data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

# PDF 파일 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)

# Chroma DB에 청크들을 벡터 임베딩으로 저장
db = Chroma(persist_directory="./chroma_korean_law", embedding_function=embedding_model)

# batch_size를 줄여서 데이터 추가 (메모리 절약)
batch_size = 5  
for i in range(0, len(docs), batch_size):
    db.add_documents(docs[i:i + batch_size])
    print(f"{i + batch_size}개 문서 처리 완료")
    
# Chroma를 Retriever로 활용
retriever = db.as_retriever() # Chroma DB가 Retriever로 변환, 임베딩 유사도로 유사 청크를 찾아냄
retriever.invoke("국회의원의 의무")

5개 문서 처리 완료
10개 문서 처리 완료
15개 문서 처리 완료
20개 문서 처리 완료
25개 문서 처리 완료
30개 문서 처리 완료
35개 문서 처리 완료
40개 문서 처리 완료
45개 문서 처리 완료
50개 문서 처리 완료
55개 문서 처리 완료


[Document(id='a4a9bdf6-46fe-4da4-a5b7-228f0229cd5c', metadata={'page': 6, 'page_label': '7', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의'),
 Document(id='496b7bbb-2169-4d84-ab89-9fa8a74b1f3c', metadata={'page': 11, 'page_label': '12', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제123조 ①국가는 농업 및 어업을 보호ㆍ육성하기 위하여 농ㆍ어촌종합개발과 그 지원등 필요한 계획을 수립ㆍ시행하\n여야 한다.\n②국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진다.\n③국가는 중소기업을 보호ㆍ육성하여야 한다.\n④국가는 농수산물의 수급균형과 유통구조의 개선에 노력하여 가격안정을 도모함으로써 농ㆍ어민의 이익을 보호\n한다.'),
 Do

In [5]:
# 좀 더 정밀하고 다양한 조정 방법
# 유사도 반환 함수 simlarity_search_with_score(), similarity_search_with_relevance_scores()
# 질문 - 유사 청크 간 거리와 유사도 점수 출력
# simlarity_search_with_score()의 출력값이 낮을 수록 질문과 유사도가 높은것, similarity_search_with_relevance_scores() 출력값이 높을 수록 질문과 유사도가 높다

result_score = db.similarity_search_with_score("대통령의 의무")
result_r_score = db._similarity_search_with_relevance_scores("대통령의 의무")
print("[유사청크 1순위]")
print(result_score[0][0].page_content)
print("\n\n[점수]")
print(result_score[0][1])
print(result_r_score[0][1])

[유사청크 1순위]
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의


[점수]
0.5362348889185601
0.6208246737368712


In [7]:
# 검색 결과 개수 조정(search_kwargs)

# 유사청크 1개만 반환
retriever = db.as_retriever(search_kwargs={"k": 1})
# retriever.get_relevant_documents("대통령의 의무")
retriever.invoke("대통령의 의무")

[Document(id='a4a9bdf6-46fe-4da4-a5b7-228f0229cd5c', metadata={'page': 6, 'page_label': '7', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의')]

In [8]:
# 검색 방식 변경(MMR 검색 유형)
# MMR(Maximal Marginal Relevance) : 문서의 유사성과 다양성을 동시에 고려하는 방법, 질문과 문서와 유사도와 문서 집합 중 가장 유사한 문서와의 유사도
# 질문과의 유사도가 높으면서 문서 집합 중 가장 유사한 문서와의 유사도가 가장 낮을 수록 MMR은 높아진다.
# MMR = 1 -> 질문과의 유사도가 가장 높은 문서 검색, MMR = 0 -> 문서 집합 중 가장 유사한 문서와의 유사도가 매우 낮은 문서 검색

# MMR 검색방식(다양성만을 고려)

retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs= {"lambda_mult": 0, "fetch_k":10, "k":3}
)

retriever.invoke("대통령의 의무")

[Document(id='a4a9bdf6-46fe-4da4-a5b7-228f0229cd5c', metadata={'page': 6, 'page_label': '7', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의'),
 Document(id='496b7bbb-2169-4d84-ab89-9fa8a74b1f3c', metadata={'page': 11, 'page_label': '12', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제123조 ①국가는 농업 및 어업을 보호ㆍ육성하기 위하여 농ㆍ어촌종합개발과 그 지원등 필요한 계획을 수립ㆍ시행하\n여야 한다.\n②국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진다.\n③국가는 중소기업을 보호ㆍ육성하여야 한다.\n④국가는 농수산물의 수급균형과 유통구조의 개선에 노력하여 가격안정을 도모함으로써 농ㆍ어민의 이익을 보호\n한다.'),
 Do

In [9]:
# 일반 유사도 검색 방식

retriever = db.as_retriever(search_kwargs={"k": 3})
retriever.invoke("대통령의 의무")

[Document(id='a4a9bdf6-46fe-4da4-a5b7-228f0229cd5c', metadata={'page': 6, 'page_label': '7', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의'),
 Document(id='496b7bbb-2169-4d84-ab89-9fa8a74b1f3c', metadata={'page': 11, 'page_label': '12', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제123조 ①국가는 농업 및 어업을 보호ㆍ육성하기 위하여 농ㆍ어촌종합개발과 그 지원등 필요한 계획을 수립ㆍ시행하\n여야 한다.\n②국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진다.\n③국가는 중소기업을 보호ㆍ육성하여야 한다.\n④국가는 농수산물의 수급균형과 유통구조의 개선에 노력하여 가격안정을 도모함으로써 농ㆍ어민의 이익을 보호\n한다.'),
 Do

MMR방식이 기존 검색 방식 대비 좀 더 다양한 청크를 유사 청크로 검색

In [10]:
# 사용자의 쿼리를 재해석해 검색하는 MultiQueryRetriever
# 사용자의 질문 내용이 벡터 DB내 청크들과 유사하지 않은 경우 LLM을 활용해 사용자 질문을 여러 버전으로 만들어 벡터 DB내 검색이 원활하게끔 한다.
# 즉, 사용자의 질문 문장을 여러개로 조합하여 파생 질문들을 만들어 DB 검색에 활용한다. 

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_google_genai import GoogleGenerativeAI 

# 질문 문장 question으로 저장
question = "대통령의 의무는 무엇이 있나요?"

# 여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = GoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", 
    api_key=os.environ["GEMINI_API_KEY"], temperature=0)

# MultiQueryRetriever에 벡터 DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever = db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging 
logging.basicConfig()
logging.getLogger("langchain.retriever.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

4

In [11]:
# 로깅을 맨 앞에 설정
import logging
# 핸들러 설정 및 포맷 정의
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
# 로거 가져오기
logger = logging.getLogger("langchain.retrievers.multi_query")
# 핸들러 추가 및 레벨 설정
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)  # DEBUG로 설정하여 더 많은 정보 출력

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_google_genai import GoogleGenerativeAI
import os

# 질문 문장 question으로 저장
question = "대통령의 의무는 무엇이 있나요?"

# 여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = GoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", 
    api_key=os.environ["GEMINI_API_KEY"], temperature=0)

# MultiQueryRetriever에 벡터 DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 생성된 질문 확인을 위해 직접 print 추가
print("Starting retrieval with question:", question)

# 여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
print(f"검색된 문서 개수: {len(unique_docs)}")

# 생성된 질문들을 확인하기 위한 코드 추가 (MultiQueryRetriever 내부 구현에 따라 달라질 수 있음)
# 가능하다면 retriever_from_llm 객체에서 생성된 질문들을 직접 접근
if hasattr(retriever_from_llm, "generated_queries"):
    print("\n생성된 질문들:")
    for i, query in enumerate(retriever_from_llm.generated_queries):
        print(f"{i+1}. {query}")

Starting retrieval with question: 대통령의 의무는 무엇이 있나요?


2025-03-06 10:42:29,557 - langchain.retrievers.multi_query - INFO - Generated queries: ['대통령의 책임은 무엇인가?', '대통령은 어떤 역할을 수행하나요?', '국가원수로서 대통령은 무슨 일을 하나요?']
INFO:langchain.retrievers.multi_query:Generated queries: ['대통령의 책임은 무엇인가?', '대통령은 어떤 역할을 수행하나요?', '국가원수로서 대통령은 무슨 일을 하나요?']


검색된 문서 개수: 4


In [2]:
# 문서를 여러 벡터로 재해석하는 MultiVectorRetriever
# 문서의 벡터를 재가공하여 검색 품질을 향상시킨다.
# 상위문서검색기(Parent Document Retriever) : 알맞은 청크 길이를 조절하는 역할
# 상위 청크 기준으로 긴 길이의 청크를 만들고 하위 청크 기준으로 짧은 길이의 청크를 만든다. 예) 상위청크 : 서시, 하위청크 : 서시의 각 행
# 검색 때는 하위 청크를 검색하고, LLM에게 컨텍스트를 전달할 때는 상위 청크를 전달한다. 

# Chroma DB에 문서 저장

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader 

loader = PyPDFLoader("./data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
docs = text_splitter.split_documents(docs)

In [3]:
# Multi Vector를 만들기 위한 작업

from langchain_community.embeddings import HuggingFaceEmbeddings # from langchain_huggingface import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings":True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma(collection_name="full_documents", embedding_function=embedding)

# 상위 문서 저장 위한 레이어 선언
store = InMemoryByteStore()
id_key = "doc_id"

# 상위 문서와 하위 문서를 연결할 키 값으로 doc_id 사용
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key
)

# 문서 id로 고유한 값을 지정하기 위해 uuid 라이브러리 호출
import uuid 
doc_ids = [str(uuid.uuid4()) for _ in docs]

C:\Users\gram\AppData\Local\Temp\ipykernel_7524\844048630.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
c:\Users\gram\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# 하위 청크로 쪼개기 위한 child_text_splitter 지정
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# 상위 청크들을 순회하며 하위 청크로 분할한 후 상위 청크 id 상속
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

#vectorstore에 하위 청크 추가
retriever.vectorstore.add_documents(sub_docs)

#docstore에 상위청크 저장할 때, doc_ids 지정
retriever.docstore.mset(list(zip(doc_ids, docs)))

: 

In [ ]:
# Vectorstore alone retrieves the small chunks
print("[하위 청크] \n")
print(retriever.vectorstore.similarity_search("국민의 권리")[0].page_content)
print("-"*50)
print("[상위 청크] \n")
print(retriever.invoke("국민의 권리")[0].page_content)

In [1]:
# 컨텍스트 재정렬, Long-Context Reorder
# Long-Context Reorder 없이 유사 문서 출력


from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_chroma import Chroma
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

Chroma().delete_collection()

# 한글 임베딩 모델 선언
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]
# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = Chroma.from_texts(texts, embedding=embedding).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.invoke(query)
docs

C:\Users\gram\AppData\Local\Temp\ipykernel_17324\1164234144.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
c:\Users\gram\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


: 

In [ ]:
# Long-Context Reorder 활용하여 유사 문서 출력

#LongContextReorder 선언
reordering = LongContextReorder()

#검색된 유사문서 중 관련도가 높은 문서를 맨앞과 맨뒤에 재정배치
reordered_docs = reordering.transform_documents(docs)
reordered_docs